PART I

In [87]:
input = "input.txt"
with open(input, 'r') as infile:
    data = [l.strip() for l in infile.readlines()]

First find the S coordinates

In [88]:
for i in range(len(data)):
    for j in range(len(data[0])):
        if data[i][j] == 'S':
            break
    if data[i][j] == 'S':
        break

In [89]:
start_i, start_j = i,j

Go from S into all four directions. Stop if:  
a) The next tile doesn't connect with the current one  
b) The next tile already has a distance assigned.

Write a function that performs one step.  
This needs to distinguish all tiles and all directions one could come from (N/S/W/E).  
Needs to return the new direction for the next tile (so the opposite of the output direction), or None if it doesn't work

In [90]:
def perform_step(current_pipe, starting_direction):
    if current_pipe == '|':
        if starting_direction == 'N':
            return 'N'
        elif starting_direction == 'S':
            return 'S'
        else: 
            return None
    if current_pipe == '-':
        if starting_direction == 'W':
            return 'W'
        elif starting_direction == 'E':
            return 'E'
        else: 
            return None
    if current_pipe == 'L':
        if starting_direction == 'N':
            return 'W'
        elif starting_direction == 'E':
            return 'S'
        else: 
            return None
    if current_pipe == 'J':
        if starting_direction == 'W':
            return 'S'
        elif starting_direction == 'N':
            return 'E'
        else: 
            return None
    if current_pipe == '7':
        if starting_direction == 'W':
            return 'N'
        elif starting_direction == 'S':
            return 'E'
        else: 
            return None
    if current_pipe == 'F':
        if starting_direction == 'E':
            return 'N'
        elif starting_direction == 'S':
            return 'W'
        else: 
            return None
    if current_pipe == '.':
        return None
assert perform_step('F', 'S') == 'W', perform_step('F', 'S')
assert perform_step('L', 'N') == 'W', perform_step('L', 'N')
assert perform_step('|', 'W') == None, perform_step('|', 'W')
assert perform_step('|', 'N') == 'N', perform_step('|', 'N')
assert perform_step('.', 'W') == None, perform_step('.', 'W')
assert perform_step('-', 'W') == 'W', perform_step('-', 'W')
assert perform_step('J', 'W') == 'S', perform_step('J', 'W')
assert perform_step('7', 'S') == 'E', perform_step('J', 'W')


Create distance map:

In [92]:
distances = [[None for j in range(len(data[0]))] for i in range(len(data))]
distances[start_i][start_j] = 0

In [93]:
#From the start, try out all four possible directions
for (x, y, direction) in [(1,0, 'N'), (0,-1, 'E'),(-1,0, 'S'), (0,1, 'W')]:
    current_distance = 1
    current_x, current_y, current_direction = (start_i + x, start_j + y, direction)
    while 0 <= current_x < len(data) and 0 <= current_y < len(data[0]) and data[current_x][current_y] != 'S':
        if not distances[current_x][current_y]:
            distances[current_x][current_y] = current_distance
        elif distances[current_x][current_y] > current_distance:
            distances[current_x][current_y] = current_distance
        else:
            break
        new_direction = perform_step(data[current_x][current_y], current_direction)
        #print(f'perform_step({data[current_x][current_y], current_direction}) ==> {new_direction}')
        if not new_direction:
            break
        elif new_direction == 'N':
            current_x += 1
        elif new_direction == 'E':
            current_y -= 1
        elif new_direction == 'S':
            current_x -= 1
        elif new_direction == 'W':
            current_y += 1
        current_distance += 1
        current_direction = new_direction
        #print(current_x, current_y, direction)

In [96]:
max(elem for row in distances for elem in row if elem)

6842

In hindsight, I'm lucky that this worked. It assumes that the element with most distance to the start is actually part of the main loop, which doesn't have to be true.

PART II:

So here we record the sequence of steps and only put it in the distance matrix if we actually end up at the start again, else we discard it.

In [213]:
input = "input.txt"
with open(input, 'r') as infile:
    data = [l.strip() for l in infile.readlines()]
for i in range(len(data)):
    for j in range(len(data[0])):
        if data[i][j] == 'S':
            break
    if data[i][j] == 'S':
        break
start_i, start_j = i,j
distances = [[None for j in range(len(data[0]))] for i in range(len(data))]
distances[start_i][start_j] = 0
#From the start, try out all four possible directions
for (x, y, direction) in [(1,0, 'N'), (0,-1, 'E'),(-1,0, 'S'), (0,1, 'W')]:
    current_sequence = []
    current_x, current_y, current_direction = (start_i + x, start_j + y, direction)
    while 0 <= current_x < len(data) and 0 <= current_y < len(data[0]) and data[current_x][current_y] != 'S':
        #if not distances[current_x][current_y]:
        #    distances[current_x][current_y] = current_distance
        #elif distances[current_x][current_y] > current_distance:
        #    distances[current_x][current_y] = current_distance
        #else:
        #    break
        current_sequence.append((current_x, current_y))
        new_direction = perform_step(data[current_x][current_y], current_direction)
        #print(f'perform_step({data[current_x][current_y], current_direction}) ==> {new_direction}')
        if not new_direction:
            break
        elif new_direction == 'N':
            current_x += 1
        elif new_direction == 'E':
            current_y -= 1
        elif new_direction == 'S':
            current_x -= 1
        elif new_direction == 'W':
            current_y += 1
        current_distance += 1
        current_direction = new_direction
    #Only add sequence to distance map if we made it back to the start
    if data[current_x][current_y] == 'S':
        for d, (x,y) in enumerate(current_sequence, start=1):
            if not distances[x][y] or distances[x][y] > d:
                distances[x][y] = d

In [214]:
for row in distances:
    print(''.join(str(tile).rjust(3) if tile is not None else ' . ' for tile in row))

 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .

Okay, that's settled. Now create a new map describing tiles inside and outside the main loop.

In [215]:
for row in data:
    print(''.join(str(tile).rjust(3) if tile else ' . ' for tile in row))

  7  .  L  J  F  F  F  7  F  7  7  .  -  -  -  -  7  F  7  7  7  7  F  |  -  F  -  7  -  7  F  7  7  L  L  7  7  J  7  7  F  |  .  |  J  F  F  7  -  F  |  7  7  .  F  F  7  7  |  F  7  7  F  -  L  |  -  L  7  F  F  L  7  L  7  -  -  .  L  |  -  -  7  L  7  .  J  -  F  7  -  |  -  F  |  7  F  J  .  F  J  J  7  F  7  -  7  -  7  J  F  L  |  7  -  |  -  7  F  J  J  7  F  F  F  7  L  L  -  -  L  7  .  -  J  J  -  F  -  F
  J  7  L  L  -  J  |  |  F  L  J  -  |  J  .  F  |  -  L  |  F  J  7  J  |  .  L  L  -  L  7  F  |  L  F  |  |  J  F  J  7  L  -  J  L  F  .  |  |  F  L  .  F  F  .  -  -  F  7  7  |  J  7  |  L  |  .  |  L  F  7  -  |  |  |  J  |  7  L  |  7  L  7  7  |  -  .  .  |  J  J  L  F  J  |  |  |  J  F  |  L  |  F  |  J  F  -  -  7  7  L  F  L  F  7  J  J  L  -  J  L  L  -  J  L  L  7  L  .  -  L  -  |  F  J  .  L  L  F  F
  L  |  -  7  L  F  L  L  7  J  |  .  .  |  F  -  7  -  7  L  |  -  7  .  L  -  -  L  -  J  L  |  .  .  F  F  J  .  |  7  |  |  F  7  F  -  -  -  -  J  J  .  

In [216]:
#Clear data from everything that's not part of the main loop
for i in range(len(data)):
    for j in range(len(data[0])):
        if not distances[i][j]:
            data[i] = data[i][:j] +  '.' + data[i][j+1:]

In [217]:
#Determine orientation of start tile
start_i, start_j
#Screw the zero check, I know Start is in the middle ;-)
if distances[start_i-1][start_j] == 1:
    if distances[start_i][start_j+1] == 1:
        start_pipe = 'L'
        data[start_i] = data[start_i][: start_j] + start_pipe + data[start_i][start_j+1:]

    elif distances[start_i+1][start_j] == 1:
        start_pipe = '|'
        data[start_i] = data[start_i][: start_j] + start_pipe + data[start_i][start_j+1:]

    elif distances[start_i][start_j-1] == 1:
        start_pipe = 'J'
        data[start_i] = data[start_i][: start_j] + start_pipe + data[start_i][start_j+1:]

elif distances[start_i][start_j+1] == 1:
    if distances[start_i+1][start_j] == 1:
        start_pipe = 'F'
        data[start_i] = data[start_i][: start_j] + start_pipe + data[start_i][start_j+1:]

    elif distances[start_i][start_j-1] == 1:
        start_pipe = '-'
        data[start_i] = data[start_i][: start_j] + start_pipe + data[start_i][start_j+1:]

elif distances[start_i+1][start_j] == 1:
    if distances[start_i][start_j-1]:
        start_pipe = '7'
        data[start_i] = data[start_i][: start_j] + start_pipe + data[start_i][start_j+1:]

In [218]:
for row in data:
    print(''.join(str(tile).rjust(3) if tile else ' . ' for tile in row))

  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  

The first idea _was_ correct in principle - count the number of times you cross the main loop.  
But it's not so easy to determine what counts as a crossing.  
We go from West to East. Therefore, if we come onto a pipe field from a non-pipe field, it must have a connection to North or South.  
Determine this, then determine the direction again when we're on a non-pipe field again. If it's the same, we don't change our side, if it's different, we do.

In [219]:
io_map = [[None for tile in data[0]] for row in data] 
for i in range(len(data)):
    inside = False
    on_pipe = False
    pipe_entry_tile = None
    pipe_exit_tile = None
    for j in range(len(data[0])):
        if data[i][j] != '.':
            #We are on the main loop
            #If it is a vertical tile, we are crossing over
            if data[i][j] == '|':
                inside = not inside
            #If it is a tile pointing to the right, we are starting a 'tangent'
            if data[i][j] in ['F', 'L']:
                pipe_entry_tile = data[i][j]
            #If it is a tile pointing to the left, we are ending a 'tangent'
            if data[i][j] in ['7', 'J']:
                pipe_exit_tile = data[i][j]
                #At the end of the tangent we need to determine if we crossed after all or not
                if pipe_entry_tile == 'F' and pipe_exit_tile == 'J':
                    inside = not inside
                if pipe_entry_tile == 'L' and pipe_exit_tile == '7':
                    inside = not inside
        else:
            io_map[i][j] = inside

In [220]:
def format_io(tile):
    if tile is None:
        return ' . '
    elif tile:
        return ' I '
    else:
        return ' O '

In [221]:
for row in io_map:
    print(''.join(format_io(tile).rjust(3) for tile in row))

 O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O 
 O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O 
 O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O

In [222]:
sum(tile for row in io_map for tile in row if tile)

393

Damn, this one was mean.